# CNN - 가위바위보

[https://www.kaggle.com/datasets/drgfreeman/rockpaperscissors](https://www.kaggle.com/datasets/drgfreeman/rockpaperscissors) 에서 배포하는 데이터셋

아래 URL을 통해 기본 파일 정리가 수행된 파일을 내려 받는다.

> [https://drive.google.com/file/d/1x6YsEBCSuxAKbmUbF-U0ntXoNwELoTVr/view?usp=sharing](https://drive.google.com/file/d/1x6YsEBCSuxAKbmUbF-U0ntXoNwELoTVr/view?usp=sharing)

## #01. 준비작업 

### [1] 패키지 참조

In [ ]:
# 연결된 모듈이 업데이트 되면 즉시 자동 로드함
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings(action="ignore")

from hossam.util import *
from hossam.plot import *
from hossam.tensor import *

import zipfile

from tensorflow.keras.preprocessing.image import ImageDataGenerator

### [2] 데이터셋 준비하기

#### (1) 파일 압축 해제

데이터셋을 캐글로부터 다운로드 받은 후 적절한 위치에 압축을 해제한다.

압축을 해제하면 cats 폴더와 dogs 폴더에 각각 5000장의 이미지가 포함되어 있다.

In [ ]:
# 압축파일의 경로
workspace_dir = "E:\\DataScience\\MainCourse\\G. 딥러닝\\02.합성곱신경망(CNN)"
file_path = os.path.join(workspace_dir, "rock-paper-scissors.zip")

# 압축을 해제할 경로
extract_dir = os.path.join(workspace_dir, "rock-paper-scissors")

# 해당 폴더가 없다면 폴더를 생성하고 파일의 압축을 해제
if not os.path.exists(extract_dir):
    os.mkdir(extract_dir)

    zip_ref = zipfile.ZipFile(file_path, "r")
    zip_ref.extractall(extract_dir)
    zip_ref.close()

#### (2) 임의의 이미지 확인

#####  이미지

실행시마다 표시 이미지가 랜덤하게 바뀐다.

In [ ]:
subdir = os.listdir(extract_dir)
subdir

In [ ]:
for s in subdir:
    path = os.path.join(extract_dir, s)
    print(path)

    image_list = os.listdir(path)
    image_count = len(image_list)

    rand = np.random.random_integers(0, image_count - 1, 5)

    fig, ax = plt.subplots(1, 5, figsize=(20, 3), dpi=100)

    for i in range(0, len(ax)):
        file_path = os.path.join(path, image_list[rand[i]])
        img = load_image(file_path)
        ax[i].imshow(img)
        ax[i].axis("off")
        ax[i].set_title(image_list[rand[i]])

    plt.show()
    plt.close()

## #02. 이미지 데이터 전처리

### [1] 이미지 전처리기 생성

In [ ]:
image_gen = ImageDataGenerator(
    rescale=1.0 / 255,  # 정규화(색상값을 0~1사이로 변경함)
    rotation_range=30,  # 이미지 무작위 회전 (30도 이내)
    width_shift_range=0.2,  # 가로 방향 이동 범위 (무작위 20% 이내)
    height_shift_range=0.2,  # 세로 방향 이동 범위 (무작위 20% 이내)
    shear_range=0.2,  # 층 밀리기 강도 (무작위 20% 이내)
    zoom_range=0.2,  # 줌 범위 (무작위 20% 이내)
    brightness_range=[0.5, 1.0],  # 이미지 밝기
    horizontal_flip=True,  # 수평 뒤집기
    vertical_flip=True,  # 수직 뒤집기
    fill_mode="nearest",  # 이미지 변형 시 채울 픽셀
    validation_split=0.2,  # 검증 데이터 비율
)

### [2] 이미지 데이터 전처리 수행

#### (1) 훈련용 이미지 데이터 생성

In [ ]:
classes = os.listdir(extract_dir)
classes

In [ ]:
train_set = image_gen.flow_from_directory(
    extract_dir,  # 이미지 파일이 위치한 폴더
    classes=classes,  # 분류할 클래스명
    batch_size=16,  # 배치 사이즈
    class_mode="categorical",  # 다항분류용임을 명시(binary or categorical)
    target_size=(64, 64),  # 변환될 이미지 해상도
    shuffle=True,  # 이미지 섞기
    color_mode="rgb",  # 컬러 이미지
    seed=get_random_state(),  # 랜덤 시드값
    subset="training",  # 훈련용 데이터 생성임을 명시
)

train_set.class_indices

#### (2) 검증용 데이터 생성

In [ ]:
test_set = image_gen.flow_from_directory(
    extract_dir,  # 이미지 파일이 위치한 폴더
    classes=classes,  # 분류할 클래스명
    batch_size=16,  # 배치 사이즈
    class_mode="categorical",  # 다항분류용임을 명시(binary or categorical)
    target_size=(64, 64),  # 변환될 이미지 해상도
    shuffle=True,  # 이미지 섞기
    color_mode="rgb",  # 컬러 이미지
    seed=get_random_state(),  # 랜덤 시드값
    subset="validation",  # 검증용 데이터 생성임을 명시
)

test_set.class_indices

## #03. 훈련 모델 적합

### [1] 모델 정의하기

In [ ]:
model = tf_create(
    layer=[
        # cnn (1)
        {
            "type": "conv2d",
            "filters": 16,
            "kernel_size": 6,
            "padding": "same",
            "input_shape": (64, 64, 3),
        },
        {"type": "batchnorm"},
        {"type": "activation", "function": "relu"},
        {"type": "maxpooling", "pool_size": (2, 2)},
        {"type": "dropout", "rate": 0.1},
        # cnn (2)
        {
            "type": "conv2d",
            "filters": 32,
            "kernel_size": 5,
            "padding": "same",
        },
        {"type": "batchnorm"},
        {"type": "activation", "function": "relu"},
        {"type": "maxpooling", "pool_size": (2, 2)},
        {"type": "dropout", "rate": 0.1},
        # cnn (3)
        {
            "type": "conv2d",
            "filters": 64,
            "kernel_size": 4,
            "padding": "same",
        },
        {"type": "batchnorm"},
        {"type": "activation", "function": "relu"},
        {"type": "maxpooling", "pool_size": (2, 2)},
        {"type": "dropout", "rate": 0.1},
        # cnn (4)
        {
            "type": "conv2d",
            "filters": 128,
            "kernel_size": 3,
            "padding": "same",
        },
        {"type": "batchnorm"},
        {"type": "activation", "function": "relu"},
        {"type": "maxpooling", "pool_size": (2, 2)},
        {"type": "dropout", "rate": 0.1},
        # 단일층
        {"type": "flatten"},
        {"type": "dense", "units": 64},
        {"type": "batchnorm"},
        {"type": "activation", "function": "relu"},
        {"type": "dense", "units": 3},  # 출력층의 수는 클래스의 수와 동일해야 함
        {"type": "batchnorm"},
        {"type": "activation", "function": "softmax"},  # 다중클래스 분류용 활성화 함수
    ],
    optimizer="adam",
    loss="categorical_crossentropy",  # 다중클래스 분류용 손실함수
    metrics=["acc"],
)

model.summary()

### [2] 학습하기

In [ ]:
%%time

batch_size = 16

result = tf_train(
    model=model,
    x_train=train_set,
    x_test=test_set,
    epochs=1000,
    steps_per_epoch=train_set.samples // batch_size,
    validation_steps=test_set.samples // batch_size,
    verbose=1
)

tf_result(result)

## #04. 학습 결과 확인

### [1] 검증 데이터의 라벨 확인

In [ ]:
y = test_set.classes
y

### [2] 검증 데이터에 대한 예측값

In [ ]:
y_pred_proba = model.predict(test_set)
y_pred_proba

In [ ]:
y_pred = np.argmax(y_pred_proba, axis=1)
y_pred

### [3] 혼동 행렬

In [ ]:
my_confusion_matrix(y, y_pred, figsize=(5, 5), dpi=100)

## #05. 학습 결과 적용

### [1] 임의의 이미지 가져오기

In [ ]:
가위_img = load_image("res/가위.jpg")
가위_img

In [ ]:
바위_img = load_image("res/바위.jpg")
바위_img

In [ ]:
보_img = load_image("res/보.jpg")
보_img

### [2] 이미지 전처리

In [ ]:
# 훈련 데이터와 동일한 크기로 리사이즈
가위_tune = tune_image(가위_img, size=(64, 64), contrast=1.5)
바위_tune = tune_image(바위_img, size=(64, 64), contrast=1.5)
보_tune = tune_image(보_img, size=(64, 64), contrast=1.5)

# 이미지 데이터 변환
가위_flow = image_gen.flow(np.array([가위_tune]))
바위_flow = image_gen.flow(np.array([바위_tune]))
보_flow = image_gen.flow(np.array([보_tune]))

# 예측값 생성
가위_pred_proba = model.predict(가위_flow)
가위_pred = np.argmax(가위_pred_proba)
print(가위_pred_proba, 가위_pred)

바위_pred_proba = model.predict(바위_flow)
바위_pred = np.argmax(바위_pred_proba)
print(바위_pred_proba, 바위_pred)

보_pred_proba = model.predict(보_flow)
보_pred = np.argmax(보_pred_proba)
print(보_pred_proba, 보_pred)